In [1]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("ReadOrdersCSV") \
    .config("spark.driver.extraClassPath", "/Users/psagar/Downloads/postgresql-42.7.3.jar") \
    .getOrCreate()


25/06/16 15:53:42 WARN Utils: Your hostname, C18952 resolves to a loopback address: 127.0.0.1; using 192.168.29.134 instead (on interface en0)
25/06/16 15:53:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/06/16 15:53:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read the CSV file with header
df_csv = spark.read.option("header", True).csv("file:///Users/psagar/fake_generation_100_rows/orders_1_100.csv")

# Show first few rows
df_csv.show()

# Optional: Print schema
df_csv.printSchema()



+---+-----------+----------------+------------+--------------------+
| id|customer_id|transaction_date|order_status|       shipping_info|
+---+-----------+----------------+------------+--------------------+
|  1|   CUST0001|      2024-10-08|  Processing|DHL Tracking #fEo...|
|  2|   CUST0002|      2024-08-02|     Shipped|FedEx Tracking #v...|
|  3|   CUST0003|      2025-02-02|   Cancelled|DHL Tracking #iaX...|
|  4|   CUST0004|      2025-05-26|     Shipped|FedEx Tracking #N...|
|  5|   CUST0005|      2024-08-03|  Processing|UPS Tracking #vqu...|
|  6|   CUST0006|      2025-03-05|   Cancelled|USPS Tracking #xv...|
|  7|   CUST0007|      2025-05-06|   Delivered|DHL Tracking #gxh...|
|  8|   CUST0008|      2024-11-10|  Processing|DHL Tracking #JOE...|
|  9|   CUST0009|      2024-07-28|  Processing|USPS Tracking #gQ...|
| 10|   CUST0010|      2025-05-06|  Processing|FedEx Tracking #u...|
| 11|   CUST0011|      2024-11-20|  Processing|UPS Tracking #CRN...|
| 12|   CUST0012|      2024-09-05|

In [3]:
# df_csv.show(100, truncate=False)


In [4]:
# Read the JSON array file using multiLine option
df_json = spark.read.option("multiLine", True).json("hdfs:///user/psagar/data/hdfs/orders_101_200.json")

# Show the data
df_json.show(20, truncate=False)

# Print schema
df_json.printSchema()


+-----------+---+------------+------------------------+----------------+
|customer_id|id |order_status|shipping_info           |transaction_date|
+-----------+---+------------+------------------------+----------------+
|CUST0101   |101|Cancelled   |USPS Tracking #IRW68534 |2024-09-24      |
|CUST0102   |102|Processing  |UPS Tracking #LUq47667  |2024-10-31      |
|CUST0103   |103|Delivered   |DHL Tracking #hKu68507  |2025-05-18      |
|CUST0104   |104|Cancelled   |USPS Tracking #Fov00065 |2024-06-24      |
|CUST0105   |105|Cancelled   |USPS Tracking #jfK95808 |2025-02-06      |
|CUST0106   |106|Processing  |DHL Tracking #vXe58952  |2024-07-05      |
|CUST0107   |107|Processing  |UPS Tracking #RwG35592  |2024-10-18      |
|CUST0108   |108|Delivered   |FedEx Tracking #dHy70269|2024-09-18      |
|CUST0109   |109|Shipped     |FedEx Tracking #nFr62157|2024-08-24      |
|CUST0110   |110|Delivered   |USPS Tracking #wnB79248 |2024-12-04      |
|CUST0111   |111|Delivered   |DHL Tracking #pml6094

In [5]:
from pyspark.sql import SparkSession

#Corrected path to your downloaded JDBC driver
jdbc_jar_path = "/Users/psagar/Downloads/postgresql-42.7.3.jar"

#Create SparkSession with JDBC support
spark = SparkSession.builder \
    .appName("PostgreSQL Table Read") \
    .config("spark.jars", jdbc_jar_path) \
    .getOrCreate()

#JDBC connection details
jdbc_url = "jdbc:postgresql://localhost:5432/task_testdb"
table_name = "public.orders_201_300"

connection_properties = {
    "user": "postgres",
    "password": "Shaan1910@@",
    "driver": "org.postgresql.Driver"
}

#Load the data from PostgreSQL into PySpark DataFrame
df_db = spark.read.jdbc(
    url=jdbc_url,
    table=table_name,
    properties=connection_properties
)

#Display top rows in Spark
df_db.show()

#Convert to Pandas DataFrame (if needed for notebook use)
# pdf = df_db.toPandas()
# pdf.head(100)


25/06/16 15:53:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
+---+-----------+----------------+------------+--------------------+
| id|customer_id|transaction_date|order_status|       shipping_info|
+---+-----------+----------------+------------+--------------------+
|201|   CUST0201|      2025-04-02|     Shipped|DHL Tracking #vHd...|
|202|   CUST0202|      2024-07-08|   Delivered|UPS Tracking #oHh...|
|203|   CUST0203|      2024-12-19|   Delivered|FedEx Tracking #o...|
|204|   CUST0204|      2025-05-16|   Delivered|UPS Tracking #AgW...|
|205|   CUST0205|      2024-11-22|  Processing|DHL Tracking #xmv...|
|206|   CUST0206|      2025-01-16|   Cancelled|USPS Tracking #gS...|
|207|   CUST0207|      2025-01-19|  Processing|DHL Tracking #maT...|
|208|   CUST0208|      2025-03-10|   Delivered|DHL Tracking #wlg...|
|209|   CUST0209|      2025-06-05|  Processing|FedEx Tracking #c...|
|210|   CUST0210|      2025-03-14|   Delivered|UPS T

In [6]:
# df_db.show(100, truncate=False)

In [7]:
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

# Clean function
def clean_df(df):
    df = df.dropDuplicates(["id"])
    df = df.dropna(subset=["id", "customer_id", "transaction_date"])
    df = df.withColumn("transaction_date", col("transaction_date").cast(DateType()))
    return df

# Use the already loaded DataFrames
df_csv_cleaned = clean_df(df_csv)
df_json_cleaned = clean_df(df_json)
df_db_cleaned = clean_df(df_db)

# Union the cleaned DataFrames
final_orders_df = df_csv_cleaned.unionByName(df_json_cleaned).unionByName(df_db_cleaned)

# Show final combined dataset
# final_orders_df.show(300, truncate=False)

# Optional: Print the schema to confirm alignment
final_orders_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- order_status: string (nullable = true)
 |-- shipping_info: string (nullable = true)



In [8]:
# final_orders_df_sorted = final_orders_df.orderBy('id')
# final_orders_df_sorted.show(300, truncate=False)
# ------------with this im getting "id" printed in lexicographical order.....as
# we want the id printed in numerical order so we are going by the script beloe

In [9]:
from pyspark.sql.functions import col

# Convert to Integer
final_orders_df = final_orders_df.withColumn("id", col("id").cast("int"))

# Then sort
final_orders_df_sorted = final_orders_df.orderBy("id")
final_orders_df_sorted.show(170, truncate=False)


+---+-----------+----------------+------------+------------------------+
|id |customer_id|transaction_date|order_status|shipping_info           |
+---+-----------+----------------+------------+------------------------+
|1  |CUST0001   |2024-10-08      |Processing  |DHL Tracking #fEo08743  |
|2  |CUST0002   |2024-08-02      |Shipped     |FedEx Tracking #vhi34386|
|3  |CUST0003   |2025-02-02      |Cancelled   |DHL Tracking #iaX18940  |
|4  |CUST0004   |2025-05-26      |Shipped     |FedEx Tracking #NIy90165|
|5  |CUST0005   |2024-08-03      |Processing  |UPS Tracking #vqu22868  |
|6  |CUST0006   |2025-03-05      |Cancelled   |USPS Tracking #xvn00808 |
|7  |CUST0007   |2025-05-06      |Delivered   |DHL Tracking #gxh97015  |
|8  |CUST0008   |2024-11-10      |Processing  |DHL Tracking #JOE42421  |
|9  |CUST0009   |2024-07-28      |Processing  |USPS Tracking #gQH21959 |
|10 |CUST0010   |2025-05-06      |Processing  |FedEx Tracking #ufy71410|
|11 |CUST0011   |2024-11-20      |Processing  |UPS 

In [30]:
from pyspark.sql.functions import expr, rand

# 1. Add a random price between 100 and 500
final_orders_df_sorted = final_orders_df_sorted.withColumn(
    "price", (rand() * 400 + 100).cast("int")  # range: 100–500
)

# 2. Add a random quantity between 1 and 5
final_orders_df_sorted = final_orders_df_sorted.withColumn(
    "quantity", (rand() * 4 + 1).cast("int")  # range: 1–5
)

# Check the updated DataFrame
final_orders_df_sorted.select("id", "customer_id", "transaction_date", "order_status", "shipping_info", "price", "quantity").show(300, truncate=False)


+---+-----------+----------------+------------+------------------------+-----+--------+
|id |customer_id|transaction_date|order_status|shipping_info           |price|quantity|
+---+-----------+----------------+------------+------------------------+-----+--------+
|1  |CUST0001   |2024-10-08      |Processing  |DHL Tracking #fEo08743  |444  |4       |
|2  |CUST0002   |2024-08-02      |Shipped     |FedEx Tracking #vhi34386|247  |2       |
|3  |CUST0003   |2025-02-02      |Cancelled   |DHL Tracking #iaX18940  |107  |1       |
|4  |CUST0004   |2025-05-26      |Shipped     |FedEx Tracking #NIy90165|200  |2       |
|5  |CUST0005   |2024-08-03      |Processing  |UPS Tracking #vqu22868  |106  |2       |
|6  |CUST0006   |2025-03-05      |Cancelled   |USPS Tracking #xvn00808 |151  |1       |
|7  |CUST0007   |2025-05-06      |Delivered   |DHL Tracking #gxh97015  |201  |2       |
|8  |CUST0008   |2024-11-10      |Processing  |DHL Tracking #JOE42421  |209  |4       |
|9  |CUST0009   |2024-07-28     

In [32]:
from pyspark.sql.functions import year, month, when, col, round


# 1. discount_percent based on transaction_date
final_orders_df_sorted = final_orders_df_sorted.withColumn(
    "discount_percent",
    when(year("transaction_date") == 2024, 30)
    .when((year("transaction_date") == 2025) & (month("transaction_date") <= 6), 20)
    .when((year("transaction_date") == 2025) & (month("transaction_date") > 6), 10)
    .otherwise(0)
)

# 2. amount_spent = price × quantity
final_orders_df_sorted = final_orders_df_sorted.withColumn(
    "amount_spent", col("price") * col("quantity")
    
)
# 3. amount_after_discount = amount_spent × (1 - discount_percent / 100)
final_orders_df_sorted = final_orders_df_sorted.withColumn(
    "amount_after_discount",
    round(col("amount_spent") * (1 - col("discount_percent") / 100), 2)
)

# 4. View final results
final_orders_df_sorted.select(
    "id", "customer_id", "transaction_date",
    "price", "quantity", "amount_spent",
    "discount_percent", "amount_after_discount"
).show(300, truncate=False)


+---+-----------+----------------+-----+--------+------------+----------------+---------------------+
|id |customer_id|transaction_date|price|quantity|amount_spent|discount_percent|amount_after_discount|
+---+-----------+----------------+-----+--------+------------+----------------+---------------------+
|1  |CUST0001   |2024-10-08      |444  |4       |1776        |30              |1243.2               |
|2  |CUST0002   |2024-08-02      |247  |2       |494         |30              |345.8                |
|3  |CUST0003   |2025-02-02      |107  |1       |107         |20              |85.6                 |
|4  |CUST0004   |2025-05-26      |200  |2       |400         |20              |320.0                |
|5  |CUST0005   |2024-08-03      |106  |2       |212         |30              |148.4                |
|6  |CUST0006   |2025-03-05      |151  |1       |151         |20              |120.8                |
|7  |CUST0007   |2025-05-06      |201  |2       |402         |20              |321

In [10]:
from pyspark.sql.functions import col, trim

# 1. Load customer JSON data from HDFS with multiline option
customer_path = "hdfs:///user/psagar/data/hdfs/customers_1_100_nested.json"
customers_df = spark.read.option("multiLine", True).json(customer_path)

# 2. Flatten nested JSON fields
customers_flat_df = customers_df.select(
    col("customer_id"),
    col("name.first_name").alias("first_name"),
    col("name.last_name").alias("last_name"),
    col("email"),
    col("phone"),
    col("address.street").alias("street"),
    col("address.city").alias("city"),
    col("address.state").alias("state"),
    col("address.postal_code").alias("postal_code"),
    col("address.country").alias("country"),
    col("created_at")
)

# 3. Clean/trim whitespace from string fields
customers_clean_df = customers_flat_df.select(
    "customer_id",
    trim(col("first_name")).alias("first_name"),
    trim(col("last_name")).alias("last_name"),
    trim(col("email")).alias("email"),
    trim(col("phone")).alias("phone"),
    trim(col("street")).alias("street"),
    trim(col("city")).alias("city"),
    trim(col("state")).alias("state"),
    trim(col("postal_code")).alias("postal_code"),
    trim(col("country")).alias("country"),
    col("created_at")
)

# 4. Show cleaned and flattened customers data to verify
customers_clean_df.show(5, truncate=False)


+-----------+----------+---------+------------------------------+--------------------+-----------------------+----------------+----------+-----------+-------------------+----------+
|customer_id|first_name|last_name|email                         |phone               |street                 |city            |state     |postal_code|country            |created_at|
+-----------+----------+---------+------------------------------+--------------------+-----------------------+----------------+----------+-----------+-------------------+----------+
|CUST0001   |Allison   |Hensley  |reevesmichael@white-parker.net|329.942.4918x349    |956 Jerry Rue          |West Kristenside|Indiana   |42875      |Mexico             |2024-10-15|
|CUST0002   |Parker    |Walls    |reedkathryn@gmail.com         |669.669.5810x89800  |421 Karen Rapid        |Denisehaven     |Wisconsin |14511      |New Zealand        |2024-08-13|
|CUST0003   |Tina      |Sutton   |lewisjamie@hunt.net           |+1-263-825-5387x1837|381 

In [11]:
pdf = customers_clean_df.toPandas()
pdf.head(5)

,customer_id,first_name,last_name,email,phone,street,city,state,postal_code,country,created_at
0,CUST0001,Allison,Hensley,reevesmichael@white-parker.net,329.942.4918x349,956 Jerry Rue,West Kristenside,Indiana,42875,Mexico,2024-10-15
1,CUST0002,Parker,Walls,reedkathryn@gmail.com,669.669.5810x89800,421 Karen Rapid,Denisehaven,Wisconsin,14511,New Zealand,2024-08-13
2,CUST0003,Tina,Sutton,lewisjamie@hunt.net,+1-263-825-5387x1837,381 Yu Meadows Apt. 948,Hansenview,Nebraska,80925,Trinidad and Tobago,2024-05-28
3,CUST0004,Marissa,Fowler,codysalazar@gmail.com,001-481-320-3964x448,996 Brown Parkway,New Emilyview,Virginia,16921,Jamaica,2023-07-03
4,CUST0005,Jeanette,Alvarado,ymunoz@collins-lin.net,(878)984-8300,08965 Holmes Burg,Oscarshire,New Jersey,02397,Jersey,2023-12-03


In [33]:
# Assuming you have final_orders_df_sorted loaded or created before this

# Join final orders with cleaned customers data on customer_id (left join)
# joined_df = final_orders_df_sorted.join(customers_clean_df, on="customer_id", how="left")

# Show combined result
# joined_df.show(5, truncate=False)


In [34]:
# pdf = joined_df.toPandas()
# pdf.head(200)

In [35]:
# Let's import the join type
from pyspark.sql.functions import col

# Join the orders with customer data
orders_customers_df = final_orders_df_sorted.join(
    customers_clean_df,
    on="customer_id",
    how="left"  # use 'inner' if you only want records that have both order and customer data
)

# Show the joined data
orders_customers_df.show(10, truncate=False)

# Optional: See the schema to understand the structure
orders_customers_df.printSchema()


+-----------+---+----------------+------------+------------------------+-----+--------+----------------+------------+---------------------+----------+---------+------------------------------+---------------------+--------------------------------+----------------+--------------+-----------+--------------------+----------+
|customer_id|id |transaction_date|order_status|shipping_info           |price|quantity|discount_percent|amount_spent|amount_after_discount|first_name|last_name|email                         |phone                |street                          |city            |state         |postal_code|country             |created_at|
+-----------+---+----------------+------------+------------------------+-----+--------+----------------+------------+---------------------+----------+---------+------------------------------+---------------------+--------------------------------+----------------+--------------+-----------+--------------------+----------+
|CUST0001   |1  |2024-10-08    

In [36]:
pdf = orders_customers_df.toPandas()
pdf.head(200)

,customer_id,id,transaction_date,order_status,shipping_info,price,quantity,discount_percent,amount_spent,amount_after_discount,first_name,last_name,email,phone,street,city,state,postal_code,country,created_at
0,CUST0001,1,2024-10-08,Processing,DHL Tracking #fEo08743,444,4,30,1776,1243.2,Allison,Hensley,reevesmichael@white-parker.net,329.942.4918x349,956 Jerry Rue,West Kristenside,Indiana,42875,Mexico,2024-10-15
1,CUST0002,2,2024-08-02,Shipped,FedEx Tracking #vhi34386,247,2,30,494,345.8,Parker,Walls,reedkathryn@gmail.com,669.669.5810x89800,421 Karen Rapid,Denisehaven,Wisconsin,14511,New Zealand,2024-08-13
2,CUST0003,3,2025-02-02,Cancelled,DHL Tracking #iaX18940,107,1,20,107,85.6,Tina,Sutton,lewisjamie@hunt.net,+1-263-825-5387x1837,381 Yu Meadows Apt. 948,Hansenview,Nebraska,80925,Trinidad and Tobago,2024-05-28
3,CUST0004,4,2025-05-26,Shipped,FedEx Tracking #NIy90165,200,2,20,400,320.0,Marissa,Fowler,codysalazar@gmail.com,001-481-320-3964x448,996 Brown Parkway,New Emilyview,Virginia,16921,Jamaica,2023-07-03
4,CUST0005,5,2024-08-03,Processing,UPS Tracking #vqu22868,106,2,30,212,148.4,Jeanette,Alvarado,ymunoz@collins-lin.net,(878)984-8300,08965 Holmes Burg,Oscarshire,New Jersey,02397,Jersey,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,CUST0196,196,2024-09-05,Processing,USPS Tracking #JZl67766,213,3,30,639,447.3,None,None,None,None,None,None,None,None,None,None
196,CUST0197,197,2024-12-08,Delivered,DHL Tracking #VhH41478,228,1,30,228,159.6,None,None,None,None,None,None,None,None,None,None
197,CUST0198,198,2025-01-13,Processing,UPS Tracking #LbI90337,391,1,20,391,312.8,None,None,None,None,None,None,None,None,None,None
198,CUST0199,199,2024-11-08,Cancelled,UPS Tracking #XoH42884,141,4,30,564,394.8,None,None,None,None,None,None,None,None,None,None


In [37]:
from pyspark.sql.functions import year, quarter, avg, round

# Step 1: Extract year and quarter
orders_summary_df = orders_customers_df.withColumn("year", year(col("transaction_date"))) \
                                       .withColumn("quarter", quarter(col("transaction_date")))

# Step 2: Group by year and quarter, then calculate average amount after discount
avg_amount_df = orders_summary_df.groupBy("year", "quarter") \
                                 .agg(
                                     round(avg("amount_after_discount"), 2).alias("avg_amount_after_discount")
                                 ) \
                                 .orderBy("year", "quarter")

# Step 3: Show the result
avg_amount_df.show()


+----+-------+-------------------------+
|year|quarter|avg_amount_after_discount|
+----+-------+-------------------------+
|2024|      2|                   578.43|
|2024|      3|                   557.84|
|2024|      4|                   579.66|
|2025|      1|                   570.22|
|2025|      2|                    560.0|
+----+-------+-------------------------+



In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("WriteToPostgres") \
    .config("spark.jars", "/Users/psagar/Downloads/postgresql-42.7.3.jar") \
    .getOrCreate()


25/06/16 15:53:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
# JDBC connection details
jdbc_url = "jdbc:postgresql://localhost:5432/task_testdb"
connection_props = {
    "user": "postgres",
    "password": "Shaan1910@@",
    "driver": "org.postgresql.Driver"
}

# Write to PostgreSQL
joined_df.write \
    .jdbc(url=jdbc_url,
          table="final_orders_customers",
          mode="append",
          properties=connection_props)


In [16]:
joined_df.groupBy("order_status").count().show()


+------------+-----+
|order_status|count|
+------------+-----+
|     Shipped|   56|
|  Processing|   87|
|   Cancelled|   77|
|   Delivered|   80|
+------------+-----+



In [17]:
joined_df.filter(joined_df["first_name"].isNull()).count()


200

In [18]:
joined_df.groupBy("country").count().orderBy("count", ascending=False).show(40)


+--------------------+-----+
|             country|count|
+--------------------+-----+
|                null|  200|
|        Saint Helena|    2|
|              Sweden|    2|
|                Peru|    2|
|           Guatemala|    2|
|            Mongolia|    2|
|Sao Tome and Prin...|    2|
| Trinidad and Tobago|    2|
|       Faroe Islands|    2|
|          Tajikistan|    2|
|Bosnia and Herzeg...|    2|
|             Bermuda|    2|
|          Micronesia|    2|
|Saint Pierre and ...|    2|
|          Madagascar|    2|
|             Nigeria|    2|
|               Spain|    2|
|            Paraguay|    1|
|             Tokelau|    1|
|      Norfolk Island|    1|
|              Angola|    1|
|French Southern T...|    1|
|           Singapore|    1|
|Northern Mariana ...|    1|
|               Qatar|    1|
|   Brunei Darussalam|    1|
|              Guyana|    1|
|              Jersey|    1|
|                Fiji|    1|
|              Greece|    1|
|Holy See (Vatican...|    1|
|             

In [19]:
from pyspark.sql.functions import split

joined_df.withColumn("carrier", split("shipping_info", " ")[0]) \
         .groupBy("carrier").count().orderBy("count", ascending=False).show()


+-------+-----+
|carrier|count|
+-------+-----+
|  FedEx|   97|
|    DHL|   71|
|    UPS|   69|
|   USPS|   63|
+-------+-----+



In [20]:
from pyspark.sql.functions import to_date, month

joined_df.groupBy(month("transaction_date").alias("month")).count().orderBy("month").show()


+-----+-----+
|month|count|
+-----+-----+
|    1|   29|
|    2|   21|
|    3|   25|
|    4|   25|
|    5|   26|
|    6|   22|
|    7|   23|
|    8|   30|
|    9|   28|
|   10|   22|
|   11|   22|
|   12|   27|
+-----+-----+



In [21]:
# joined_df.groupBy("customer_id").count().filter("count > 1").show()


In [22]:
# joined_df.filter(joined_df["shipping_info"].isNull()).count()


In [23]:
joined_df.select("customer_id", "created_at").orderBy("created_at").show(5)


+-----------+----------+
|customer_id|created_at|
+-----------+----------+
|   CUST0101|      null|
|   CUST0102|      null|
|   CUST0103|      null|
|   CUST0104|      null|
|   CUST0105|      null|
+-----------+----------+
only showing top 5 rows



In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, datediff

# Define a window partitioned by customer_id and ordered by transaction_date
window_spec = Window.partitionBy("customer_id").orderBy("transaction_date")

# Use lag to get the previous order date for each customer
orders_with_lag = joined_df.withColumn(
    "prev_order_date",
    lag("transaction_date").over(window_spec)
)

# Calculate the difference in days between current and previous order
orders_with_diff = orders_with_lag.withColumn(
    "days_between",
    datediff(col("transaction_date"), col("prev_order_date"))
)

# Filter to keep only consecutive orders (difference = 1 day)
consecutive_orders = orders_with_diff.filter(col("days_between") == 1)

# Show results
consecutive_orders.select(
    "customer_id", "id", "transaction_date", "prev_order_date", "days_between"
).orderBy("customer_id", "transaction_date").show()


+-----------+---+----------------+---------------+------------+
|customer_id| id|transaction_date|prev_order_date|days_between|
+-----------+---+----------------+---------------+------------+
+-----------+---+----------------+---------------+------------+



In [25]:
from pyspark.sql.functions import count

# Group by customer_id and count number of orders
repetitive_customers = joined_df.groupBy("customer_id") \
    .agg(count("id").alias("order_count")) \
    .filter("order_count > 1")

# Show repetitive customers
repetitive_customers.show()


+-----------+-----------+
|customer_id|order_count|
+-----------+-----------+
+-----------+-----------+



In [26]:
from pyspark.sql.functions import max

latest_orders = joined_df.groupBy("customer_id") \
    .agg(max("transaction_date").alias("latest_order_date"))

latest_orders.show()


+-----------+-----------------+
|customer_id|latest_order_date|
+-----------+-----------------+
|   CUST0015|       2025-05-24|
|   CUST0004|       2025-05-26|
|   CUST0097|       2024-12-26|
|   CUST0052|       2024-08-23|
|   CUST0057|       2025-04-09|
|   CUST0010|       2025-05-06|
|   CUST0044|       2024-10-26|
|   CUST0064|       2025-01-26|
|   CUST0093|       2024-09-20|
|   CUST0080|       2024-08-27|
|   CUST0077|       2025-01-18|
|   CUST0087|       2024-11-29|
|   CUST0007|       2025-05-06|
|   CUST0042|       2025-03-23|
|   CUST0017|       2025-02-12|
|   CUST0019|       2024-09-11|
|   CUST0038|       2025-03-04|
|   CUST0055|       2024-10-17|
|   CUST0095|       2024-11-22|
|   CUST0035|       2024-07-24|
+-----------+-----------------+
only showing top 20 rows



In [27]:
# import os
# import shutil

# # File and archive directory paths
# local_file = "/Users/psagar/fake_generation_100_rows/orders_1_100.csv"
# archive_dir = "/Users/psagar/fake_generation_100_rows/archive/"

# # Create archive directory if it doesn't exist
# os.makedirs(archive_dir, exist_ok=True)

# # Move the file
# shutil.move(local_file, os.path.join(archive_dir, os.path.basename(local_file)))

# print("Local file archived successfully.")


In [28]:
# import subprocess

# # File paths in HDFS
# orders_file_hdfs = "/user/psagar/data/hdfs/orders_101_200.json"
# customers_file_hdfs = "/user/psagar/data/hdfs/customers_1_100_nested.json"
# archive_dir_hdfs = "/user/psagar/data/hdfs/archive/"

# # Create HDFS archive directory if it doesn't exist
# subprocess.run(["hdfs", "dfs", "-mkdir", "-p", archive_dir_hdfs], check=True)

# # Move the files to archive
# subprocess.run(["hdfs", "dfs", "-mv", orders_file_hdfs, archive_dir_hdfs], check=True)
# subprocess.run(["hdfs", "dfs", "-mv", customers_file_hdfs, archive_dir_hdfs], check=True)

# print("HDFS files archived successfully.")
